# Musicians- Medium

## 6.
**List the names, dates of birth and the instrument played of living musicians who play a instrument which Theo also plays.**

## 7.
**List the name and the number of players for the band whose number of players is greater than the average number of players in each band.**

## 8.
**List the names of musicians who both conduct and compose and live in Britain.**

## 9.
**Show the least commonly played instrument and the number of musicians who play it.**

## 10.
**List the bands that have played music composed by Sue Little; Give the titles of the composition in each case.**